# Import

In [1]:
import json
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta

# function to download stock data

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # Write heading of the columns
        f.write('Date,Open,High,Low,Close,Volume\n')
        # load share data for the time period needed
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Choose the right columns
        stock_data.to_csv(f, header=False)  # Write the data to the file

# Define the shares

In [3]:
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']

# Read start and end for the sequence, which will be predicted from Excel

In [4]:
start_date = '2019-01-01'
end_date = '2019-12-31'


# Read close values at the begin of the time period

In [5]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = start_date_obj - timedelta(days=1)

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2018-12-27 00:00:00
2018-12-31 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)

print(combined_data_begin)

     Symbol  Close_Value
0    ALV.DE   175.139999
1    DBK.DE     6.967000
2   VOW3.DE   138.919998
3    BMW.DE    70.699997
4    ADS.DE   182.399994
5    BEI.DE    91.160004
6    DTE.SG    13.690000
7    SAP.DE    86.930000
8   1COV.DE    43.180000
9    BAS.DE    60.400002
10  EOAN.DE     8.627000
11   RWE.DE    18.965000


# read predictions

In [7]:
# Define the name of the file
json_file = 'settings/predictions1Year_dict.json'

# read array from the json file
with open(json_file, 'r') as f:
    predictions_dict = json.load(f)
    
# Convert predictions_dict to DataFrame
predictions_df = pd.DataFrame(predictions_dict)
print(predictions_df)

                             ALV.DE     DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead  50.421959  14.869346  29.790468  39.409405   

                             ADS.DE   BEI.DE     DTE.SG     SAP.DE    1COV.DE  \
prediction_20_days_ahead  71.390404  23.6108  15.274905  31.452709  12.450314   

                             BAS.DE    EOAN.DE     RWE.DE  
prediction_20_days_ahead  41.113701  11.716145  18.942484  


# Calculate return in percentage

In [8]:
# Für jedes Symbol in predictions_df den entsprechenden Close_Value aus combined_data_begin abziehen
for symbol in predictions_df.columns:
    if symbol in combined_data_begin['Symbol'].values:
        close_value = combined_data_begin.loc[combined_data_begin['Symbol'] == symbol, 'Close_Value'].values[0]
        predictions_df[symbol] = ((predictions_df[symbol] - close_value) / close_value) * 100

print(predictions_df)   

                             ALV.DE      DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead -71.210484  113.425372 -78.555666 -44.258265   

                             ADS.DE     BEI.DE     DTE.SG     SAP.DE  \
prediction_20_days_ahead -60.860523 -74.099606  11.577105 -63.818349   

                            1COV.DE     BAS.DE    EOAN.DE    RWE.DE  
prediction_20_days_ahead -71.166481 -31.930961  35.807868 -0.118725  


# load share value from 2019

In [9]:
start_date = start_date
end_date = end_date
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

Startdatum: 2019-01-01, Enddatum: 2019-12-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [10]:
# list for the las close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
print(last_close_values[0][1])
# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

218.3999938964844
Liste der letzten Close-Werte:
ALV.DE: 218.3999938964844
DBK.DE: 6.916999816894531
VOW3.DE: 176.24000549316406
BMW.DE: 73.13999938964844
ADS.DE: 289.79998779296875
BEI.DE: 106.6500015258789
DTE.SG: 13.6899995803833
SAP.DE: 120.31999969482422
1COV.DE: 41.45000076293945
BAS.DE: 67.3499984741211
EOAN.DE: 9.52400016784668
RWE.DE: 27.350000381469727


# Historic data from 2018 for the covarianz matrix

In [11]:
# Definition of the time period
start_date2018 = '2018-01-01'
end_date2018 = '2018-12-31'
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2018-12-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file 

In [12]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)
    
# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE     DBK.DE     VOW3.DE     BMW.DE      ADS.DE  \
2018-01-02  192.699997  15.958000  165.699997  86.400002  167.149994   
2018-01-03  193.000000  15.910000  171.440002  86.860001  168.050003   
2018-01-04  195.500000  16.332001  174.440002  87.480003  170.250000   
2018-01-05  198.559998  15.490000  179.199997  88.500000  172.050003   
2018-01-08  198.860001  15.340000  179.839996  89.669998  172.750000   
...                ...        ...         ...        ...         ...   
2018-12-19  175.940002   7.530000  146.880005  73.449997  187.050003   
2018-12-20  174.960007   7.000000  143.899994  71.839996  183.750000   
2018-12-21  175.020004   7.042000  143.300003  71.930000  184.750000   
2018-12-27  172.160004   6.750000  137.440002  69.860001  180.100006   
2018-12-28  175.139999   6.967000  138.919998  70.699997  182.399994   

                BEI.DE  DTE.SG     SAP.DE    1COV.DE     BAS.DE  EOAN.DE  \
2018-01-02   96.199997  14.872  92.800003  85.639999  91.30

# Mean variance optimization

In [13]:
def mean_variance_optimization(expected_returns, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # start weighs
    bounds = [(0, 1)] * n  # weigh border (0-100% for each share)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x


# Portfilio allocation

In [14]:
def allocate_portfolio(predictions, historical_data, initial_capital):
    # Calculate expected returns based on the predictions
    expected_returns = predictions.mean()
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation


# Calculate portfolio

In [15]:
initial_capital = 1000 # 1000€ start capital
print(initial_capital)
# Calculate the portfolio allocation
portfolio_allocation = allocate_portfolio(predictions_df, combined_historical2018_data, initial_capital)
# print portfolio allocation
print(portfolio_allocation)

1000
[1.13166176e-07 1.51256776e+02 3.30266124e-07 1.50661152e-07
 8.93798226e+00 1.47354927e-05 1.81694249e-04 1.80577688e-05
 6.40847335e-07 3.88575039e-07 8.39805823e+02 1.28824550e-04]


# Simulation for one year

In [16]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

6.461469458945479e-10
1.1316617570959432e-07
1.1316617570959432e-07
-----------------------------
21.710460134700476
151.25677592409588
151.25677603726206
-----------------------------
2.377383592489947e-09
3.30266124315584e-07
151.25677636752818
-----------------------------
2.1309923466263185e-09
1.5066115240320817e-07
151.25677651818933
-----------------------------
0.04900209738686987
8.937982264279999
160.19475878246934
-----------------------------
1.6164427473497068e-07
1.473549267679894e-05
160.194773517962
-----------------------------
1.3272041981306533e-05
0.00018169424915491598
160.19495521221117
-----------------------------
2.0772769724633339e-07
1.8057768785017224e-05
160.19497326997995
-----------------------------
1.4841299924240664e-08
6.408473352579171e-07
160.1949739108273
-----------------------------
6.433361415471675e-09
3.8857503931101965e-07
160.19497429940233
-----------------------------
97.34621965590765
839.8058228603537
1000.0007971597561
-----------------

In [17]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

218.3999938964844
6.461469458945479e-10
1.4111848903960129e-07
----------------------------
6.916999816894531
21.710460134700476
150.1712489175377
----------------------------
176.24000549316406
2.377383592489947e-09
150.17124933652778
----------------------------
73.13999938964844
2.1309923466263185e-09
150.17124949238857
----------------------------
289.79998779296875
0.04900209738686987
164.37205671693332
----------------------------
106.6500015258789
1.6164427473497068e-07
164.37207395629545
----------------------------
13.6899995803833
1.3272041981306533e-05
164.37225565054462
----------------------------
120.31999969482422
2.0772769724633339e-07
164.3722806443411
----------------------------
41.45000076293945
1.4841299924240664e-08
164.372281259513
----------------------------
67.3499984741211
6.433361415471675e-09
164.37228169279987
----------------------------
9.52400016784668
97.34621965590765
1091.497694034904
----------------------------
27.350000381469727
6.7927523757088236